In [1]:
# generate and human pick valid puzzle mete data
import random
from tqdm.notebook import tqdm
from all_puzzle_settings import *

# Function operator puzzle

In [2]:
def check_repeated_elementary_function(term):
    terms = term.split('+')
    check_all_constant_count = 0
    for single_term in terms:
        check_all_constant_count += len(single_term)
    if check_all_constant_count == len(terms):
        return False
    cleaned_terms = []
    for tem_term in terms:
        term_split = tem_term.split('*')
        if len(term_split) == 1:
            cleaned_terms.append(term_split[0])
        else:
            cleaned_terms.append(term_split[1])
    return len(set(cleaned_terms)) == len(cleaned_terms) # return false if have repeatation in this function
    
def generate_random_function(num_function,term_per_function,number_of_elementary_function,num_unknown_parameter,random_seed = None):
    if random_seed is not None:
        random.seed(random_seed)
    elementary_function_list = ['x^0','x','|x|','-x','1/x','sin(x)','x^2']
    parameter_list = list(range(1,10))
    # num_function = 3
    # term_per_function = 2
    # number_of_elementary_function = 2
    # num_unknown_parameter = 4
    total_num_terms = int(num_function*term_per_function)
    assert total_num_terms == num_function*term_per_function, f'''changed the terms'''
    assert num_unknown_parameter <= 10,f'''Num parameter should less than 10'''
    assert num_unknown_parameter <= total_num_terms,f'''Num parameters should less than total num of terms'''
    assert 2*num_unknown_parameter >= total_num_terms,f'''2*Num of parameters should more than total number of terms'''

    sample_elementart_functions = random.sample(elementary_function_list, number_of_elementary_function)
    sample_parameters = random.sample(parameter_list,num_unknown_parameter)

    if total_num_terms >= len(sample_parameters):
        all_parameters = random.sample(sample_parameters, len(sample_parameters))
        all_parameters.extend(random.sample(sample_parameters, total_num_terms - len(sample_parameters)))
    else:
        assert False, f'''total number of terms should be more than num of unknown parameters'''

    all_elementary_functions = []
    while len(all_elementary_functions) < total_num_terms:
        all_elementary_functions.extend(random.sample(sample_elementart_functions, len(sample_elementart_functions)))

    all_elementary_functions = all_elementary_functions[:total_num_terms]
    functions = []
    drop_this_random_seed = False # check if each function have same elementary functions in different terms
    if num_function == 1:
        cur_function = []
        for item in zip(all_parameters, all_elementary_functions):
            cur_parameter = item[0]
            cur_elementary_function = item[1]
            if cur_elementary_function == 'x^0':
                cur_term = str(cur_parameter)
            else:
                cur_term = str(cur_parameter) + '*' +cur_elementary_function
            cur_function.append(cur_term)
        functions.append('+'.join(cur_function))
    else:
        function_count_index = random.sample(list(range(total_num_terms-1)), num_function-1) # split into segments
        function_count_index.sort()
        cur_function = []
        cur_index_count = 0
        for item in zip(all_parameters, all_elementary_functions):
            cur_parameter = item[0]
            cur_elementary_function = item[1]
            if cur_elementary_function == 'x^0':
                cur_term = str(cur_parameter)
            else:
                cur_term = str(cur_parameter) + '*' +cur_elementary_function
            cur_function.append(cur_term)
            if cur_index_count in function_count_index:
                functions.append('+'.join(cur_function))
                cur_function = []
            cur_index_count += 1 
            
        functions.append('+'.join(cur_function))
        
    for tem_function in functions:
        drop_this_random_seed = drop_this_random_seed or not check_repeated_elementary_function(tem_function)
    if drop_this_random_seed:
        return None
    else:
        return functions, sample_parameters, sample_elementart_functions


In [3]:
# Get all previous hand made functions
hand_mand_functions = {'round1':[]} # Each round is consist of 20 puzzles
for item in Function_operator_puzzles['Level1'].items():
    function_list = item[1]['Space_items']['Puzzle room'][0]['Special_label'].split(',')[1:]
    function_list = [i.replace(' ','') for i in function_list]
    function_list = frozenset(function_list)
    hand_mand_functions['round1'].append(function_list)
def get_all_existing_functions(function_dict):
    all_functions = []
    for item in function_dict.items():
        all_functions.extend(item[1])
    print(f'Currently there are {len(set(all_functions))} unique function combinations')
    return set(all_functions)
get_all_existing_functions(hand_mand_functions)

Currently there are 20 unique function combinations


{frozenset({'2*x+4', '2*x^2+7*x+9', '9*sin(x)'}),
 frozenset({'1*sin(x)+2*1/x', '2*x^2+3*1/x', '3'}),
 frozenset({'1*x'}),
 frozenset({'3*x+2', '4*sin(x)+2*1/x+3', '4*x^2'}),
 frozenset({'6*sin(x)+9*x^2', '9'}),
 frozenset({'1*sin(x)+5*x', '1*|x|', '5*1/x'}),
 frozenset({'7*1/x'}),
 frozenset({'3*x^2', '9*sin(x)'}),
 frozenset({'4*1/x', '6*x^2'}),
 frozenset({'6'}),
 frozenset({'7*x^2'}),
 frozenset({'2*x^2+7*x+9', '4*|x|+7', '7*1/x'}),
 frozenset({'2*1/x', '2*x+3*1/x'}),
 frozenset({'2*sin(x)'}),
 frozenset({'4*sin(x)', '9*x'}),
 frozenset({'2*sin(x)', '9*x^2+2*x', '9*|x|+5*x'}),
 frozenset({'1*sin(x)+4*1/x', '4*1/x'}),
 frozenset({'4*1/x', '6*x^2+4*1/x'}),
 frozenset({'2*|x|', '7*sin(x)'}),
 frozenset({'1', '4*|x|'})}

In [4]:
def pick_difficult_puzzle(round_index, start_seed, all_puzzle_dict, difficulty_parameter_list, difficulty_name):
    difficulty_1_puzzle_count = 0
    difficulty_1_puzzle_number = difficulty_parameter_list[4]
    while difficulty_1_puzzle_count < difficulty_1_puzzle_number:
        num_function = difficulty_parameter_list[0]
        num_unknown_parameter = difficulty_parameter_list[1]
        avg_term_per_function = difficulty_parameter_list[2]
        number_of_elementary_function = difficulty_parameter_list[3]
        functions = generate_random_function(num_function,
                                             avg_term_per_function,
                                             number_of_elementary_function,
                                             num_unknown_parameter,
                                             random_seed = start_seed)
        # check if generated function already exist
        if functions is None:
            start_seed += 1
            continue
        function = frozenset(functions[0])
        all_functions = get_all_existing_functions(all_puzzle_dict)
        if function not in all_functions and function is not None:
            print(function,f'This is a difficulty {difficulty_name} puzzle')
            add_bool = int(input('Input 1 if you want this puzzle, Input 0 if you want to drop this puzzle'))
            if add_bool == 1:
                all_puzzle_dict[f'round{round_index}'].append(function)
                difficulty_1_puzzle_count += 1 
        start_seed += 1
def generate_new_round_functions(round_index, start_seed, all_puzzle_dict):
    random.seed(start_seed)
    random_seed = start_seed
    all_puzzle_dict[f'round{round_index}'] = []
    # start generate difficulty_1 puzzles:
    puzzle_generate_parameters = [1,1,1,1,5] # No. functions, No. unknown parameters, Avg tem per function, No. elementary functions, Total puzzles_needed
    pick_difficult_puzzle(round_index, start_seed, all_puzzle_dict,puzzle_generate_parameters, 1)
    puzzle_generate_parameters = [2,2,1,2,5] # No. functions, No. unknown parameters, Avg tem per function, No. elementary functions, Total puzzles_needed
    pick_difficult_puzzle(round_index, start_seed, all_puzzle_dict,puzzle_generate_parameters, 2)
    puzzle_generate_parameters = [2,2,1.5,2,5] # No. functions, No. unknown parameters, Avg tem per function, No. elementary functions, Total puzzles_needed
    pick_difficult_puzzle(round_index, start_seed, all_puzzle_dict,puzzle_generate_parameters, 3)
    puzzle_generate_parameters = [3,3,5/3,4,3] # No. functions, No. unknown parameters, Avg tem per function, No. elementary functions, Total puzzles_needed
    pick_difficult_puzzle(round_index, start_seed, all_puzzle_dict,puzzle_generate_parameters, 4)
    puzzle_generate_parameters = [3,4,2,5,2] # No. functions, No. unknown parameters, Avg tem per function, No. elementary functions, Total puzzles_needed
    pick_difficult_puzzle(round_index, start_seed, all_puzzle_dict,puzzle_generate_parameters, 5)

| Puzzle ID    | No. of Functions | No. of Unknown Parameters | Total   No. term| No. of Elementary Functions |
|--------------|------------------|---------------------------|-----------------|-----------------------------|
| 1–25   5+(20)| 1                | 1                         | 1               | 1                           |
| 26–50  5+(20)| 2                | 2                         | 2               | 2                           |
| 51–75  5+(20)| 2                | 2                         | 3               | 2                           |
| 76–100 3+(12)| 3                | 3                         | 5               | 4                           |
| 76–100 2+(8) | 3                | 4                         | 6               | 5                           |

In [ ]:
generate_new_round_functions(2, 0, hand_mand_functions)

In [ ]:
generate_new_round_functions(3, 1000, hand_mand_functions)

In [ ]:
generate_new_round_functions(4, 2000, hand_mand_functions)

In [ ]:
generate_new_round_functions(5, 3000, hand_mand_functions)

In [13]:
import pickle

# with open('Function_operator_puzzle_functions.pkl', 'wb') as file:
#     pickle.dump(hand_mand_functions, file)

{'round1': [frozenset({'6'}),
  frozenset({'1*x'}),
  frozenset({'2*sin(x)'}),
  frozenset({'7*x^2'}),
  frozenset({'7*1/x'}),
  frozenset({'1', '4*|x|'}),
  frozenset({'2*|x|', '7*sin(x)'}),
  frozenset({'3*x^2', '9*sin(x)'}),
  frozenset({'4*sin(x)', '9*x'}),
  frozenset({'4*1/x', '6*x^2'}),
  frozenset({'4*1/x', '6*x^2+4*1/x'}),
  frozenset({'1*sin(x)+5*x', '1*|x|', '5*1/x'}),
  frozenset({'1*sin(x)+4*1/x', '4*1/x'}),
  frozenset({'2*1/x', '2*x+3*1/x'}),
  frozenset({'6*sin(x)+9*x^2', '9'}),
  frozenset({'1*sin(x)+2*1/x', '2*x^2+3*1/x', '3'}),
  frozenset({'2*sin(x)', '9*x^2+2*x', '9*|x|+5*x'}),
  frozenset({'3*x+2', '4*sin(x)+2*1/x+3', '4*x^2'}),
  frozenset({'2*x^2+7*x+9', '4*|x|+7', '7*1/x'}),
  frozenset({'2*x+4', '2*x^2+7*x+9', '9*sin(x)'})],
 'round2': [frozenset({'2*x'}),
  frozenset({'1*x^2'}),
  frozenset({'5*1/x'}),
  frozenset({'2*x^2'}),
  frozenset({'6*-x'}),
  frozenset({'1*-x', '7*x^2'}),
  frozenset({'2*1/x', '5*x'}),
  frozenset({'3*x', '9*1/x'}),
  frozenset({'2*|x

# Escape room

In [3]:
# Make sure at least one paintings for each type
All_paintings = {
    'oil':{'Blue': [44,49,50,51],
           'Yellow': [45,47],
           'Black': [46],
           'Green': [48]},
    'acrylic': {'Blue':[56,57],
                'Yellow':[53],
                'Black':[52],
                'Green':[54,55]},
    'watercolor': {'Blue': [60,61,62],
                   'Yellow':[58],
                   'Green':[59],
                   'Black':[]}
}

existing_combination = {'round1':[]}
for i in range(1,11):
    paints = Art_gallery_puzzles['Level1'][f'puzzle{i}']['Space_items']['Corridor'][1:]
    existing_combination['round1'].append(frozenset(paints))
    
def get_all_combination_paintings(all_paint_dict):
    all_paints = []
    for item in existing_combination.items():
        all_paints.extend(item[1])
    return set(all_paints)

# 40 remaining
# select 3 from each type first
def random_select_paints(existing_combination, all_paints_needed, round_index, random_seed):
    random.seed(random_seed)
    select_paints = []
    oil_paints = []
    for color in All_paintings['oil']:
        oil_paints.extend(All_paintings['oil'][color])
    random_index = random.sample(list(range(len(oil_paints))),1)[0]
    select_paints.append(oil_paints.pop(random_index))

    acrylic_paints = []
    for color in All_paintings['acrylic']:
        acrylic_paints.extend(All_paintings['acrylic'][color])
    random_index = random.sample(list(range(len(acrylic_paints))),1)[0]
    select_paints.append(acrylic_paints.pop(random_index))

    watercolor_paints = []
    for color in All_paintings['watercolor']:
        watercolor_paints.extend(All_paintings['watercolor'][color])
    random_index = random.sample(list(range(len(watercolor_paints))),1)[0]
    select_paints.append(watercolor_paints.pop(random_index))
    
    remaining_paints = []
    remaining_paints.extend(oil_paints)
    remaining_paints.extend(acrylic_paints)
    remaining_paints.extend(watercolor_paints)
    select_paints.extend(random.sample(remaining_paints, all_paints_needed-3))
    if frozenset(select_paints) not in get_all_combination_paintings(existing_combination):
        existing_combination[f'round{round_index}'].append(frozenset(select_paints))
        return 1
    else:
        print('Generated combination already exist, please change a seed')
        return 0

In [5]:
existing_combination

{'round1': [frozenset({50, 56, 59}),
  frozenset({47, 50, 54, 58, 61}),
  frozenset({48, 49, 53, 57, 61, 62}),
  frozenset({45, 47, 49, 51, 55, 56, 62}),
  frozenset({50, 52, 54, 55, 57, 60, 61, 62}),
  frozenset({44, 48, 49, 53, 56, 57, 59, 60, 62}),
  frozenset({45, 49, 51, 52, 53, 55, 56, 58, 61, 62}),
  frozenset({45, 47, 49, 53, 54, 55, 56, 57, 58, 60, 62}),
  frozenset({44, 46, 49, 50, 52, 53, 55, 56, 57, 58, 61, 62}),
  frozenset({44, 47, 48, 50, 51, 52, 53, 57, 58, 59, 60, 61, 62})]}

In [6]:
round_index = 2
num_paints = [3,5,6,7,8,9,10,11,12,13]
start_seed = 0
existing_combination[f'round{round_index}'] = []
for i in num_paints:
    success_status = random_select_paints(existing_combination, i, round_index, start_seed)
    start_seed += 1
    while success_status == 0:
        success_status = random_select_paints(existing_combination, i, round_index, start_seed)
        start_seed += 1

In [7]:
round_index = 3
num_paints = [3,5,6,7,8,9,10,11,12,13]
start_seed = 1000
existing_combination[f'round{round_index}'] = []
for i in num_paints:
    success_status = random_select_paints(existing_combination, i, round_index, start_seed)
    start_seed += 1
    while success_status == 0:
        success_status = random_select_paints(round)
        start_seed += 1

In [8]:
round_index = 4
num_paints = [3,5,6,7,8,9,10,11,12,13]
start_seed = 2000
existing_combination[f'round{round_index}'] = []
for i in num_paints:
    success_status = random_select_paints(existing_combination, i, round_index, start_seed)
    start_seed += 1
    while success_status == 0:
        success_status = random_select_paints(round)
        start_seed += 1

In [9]:
round_index = 5
num_paints = [3,5,6,7,8,9,10,11,12,13]
start_seed = 3000
existing_combination[f'round{round_index}'] = []
for i in num_paints:
    success_status = random_select_paints(existing_combination, i, round_index, start_seed)
    start_seed += 1
    while success_status == 0:
        success_status = random_select_paints(round)
        start_seed += 1

In [11]:
existing_combination

{'round1': [frozenset({50, 56, 59}),
  frozenset({47, 50, 54, 58, 61}),
  frozenset({48, 49, 53, 57, 61, 62}),
  frozenset({45, 47, 49, 51, 55, 56, 62}),
  frozenset({50, 52, 54, 55, 57, 60, 61, 62}),
  frozenset({44, 48, 49, 53, 56, 57, 59, 60, 62}),
  frozenset({45, 49, 51, 52, 53, 55, 56, 58, 61, 62}),
  frozenset({45, 47, 49, 53, 54, 55, 56, 57, 58, 60, 62}),
  frozenset({44, 46, 49, 50, 52, 53, 55, 56, 57, 58, 61, 62}),
  frozenset({44, 47, 48, 50, 51, 52, 53, 57, 58, 59, 60, 61, 62})],
 'round2': [frozenset({46, 52, 60}),
  frozenset({49, 50, 54, 57, 60}),
  frozenset({44, 51, 55, 56, 60, 62}),
  frozenset({46, 47, 51, 53, 54, 56, 59}),
  frozenset({49, 50, 51, 53, 56, 59, 60, 61}),
  frozenset({44, 45, 51, 52, 55, 56, 60, 61, 62}),
  frozenset({44, 46, 49, 50, 51, 52, 53, 54, 56, 62}),
  frozenset({44, 46, 47, 49, 52, 53, 57, 58, 59, 61, 62}),
  frozenset({44, 45, 47, 49, 50, 51, 52, 53, 55, 57, 58, 62}),
  frozenset({44, 47, 48, 49, 50, 52, 54, 56, 57, 58, 59, 61, 62})],
 'roun

In [13]:
import pickle

# with open('Escape_room_puzzle_paints.pkl', 'wb') as file:
#     pickle.dump(existing_combination, file)

with open('Escape_room_puzzle_paints.pkl', 'rb') as file:
    existing_combination = pickle.load(file)
existing_combination

{'round1': [frozenset({50, 56, 59}),
  frozenset({47, 50, 54, 58, 61}),
  frozenset({48, 49, 53, 57, 61, 62}),
  frozenset({45, 47, 49, 51, 55, 56, 62}),
  frozenset({50, 52, 54, 55, 57, 60, 61, 62}),
  frozenset({44, 48, 49, 53, 56, 57, 59, 60, 62}),
  frozenset({45, 49, 51, 52, 53, 55, 56, 58, 61, 62}),
  frozenset({45, 47, 49, 53, 54, 55, 56, 57, 58, 60, 62}),
  frozenset({44, 46, 49, 50, 52, 53, 55, 56, 57, 58, 61, 62}),
  frozenset({44, 47, 48, 50, 51, 52, 53, 57, 58, 59, 60, 61, 62})],
 'round2': [frozenset({46, 52, 60}),
  frozenset({49, 50, 54, 57, 60}),
  frozenset({44, 51, 55, 56, 60, 62}),
  frozenset({46, 47, 51, 53, 54, 56, 59}),
  frozenset({49, 50, 51, 53, 56, 59, 60, 61}),
  frozenset({44, 45, 51, 52, 55, 56, 60, 61, 62}),
  frozenset({44, 46, 49, 50, 51, 52, 53, 54, 56, 62}),
  frozenset({44, 46, 47, 49, 52, 53, 57, 58, 59, 61, 62}),
  frozenset({44, 45, 47, 49, 50, 51, 52, 53, 55, 57, 58, 62}),
  frozenset({44, 47, 48, 49, 50, 52, 54, 56, 57, 58, 59, 61, 62})],
 'roun

# Reactor

In [49]:
hand_made_strings = {'round1':[]}
for puzzle_index in range(1,6):
    target_string = Reactor_puzzles['Level1'][f'puzzle{puzzle_index}']['Space_items']['Chemical laboratory'][1]['Special_label']
    hand_made_strings['round1'].append(target_string)
hand_made_strings
def get_all_strings(all_string_dict):
    all_string = []
    for item in all_string_dict.items():
        all_string.extend(item[1])
    return set(all_string)
hand_made_strings

{'round1': ['ACB', 'CCADD', 'CADEA', 'ABCDEF', 'FEADE']}

In [50]:
def generate_random_strings(hand_made_strings, start_seed, round_index):
    hand_made_strings[f'round{round_index}'] = []
    string_len = [3,5,5,5,6]
    elemental_letter = ['A', 'B', 'C', 'D', 'E', 'F']
    example_string = ['CAB', 'BCA', 'CBA', 'AAA', 'BDAC'] # These samples are used in the prompt as examples
    for cur_string_len in string_len:
        random.seed(start_seed)
        generated_string = ''.join(random.choices(elemental_letter,k=cur_string_len))
        start_seed += 1
        while generated_string in get_all_strings(hand_made_strings):
            print(f'{generated_string} already exist')
            random.seed(start_seed)
            generated_string = ''.join(random.choices(elemental_letter,k=cur_string_len))
            start_seed += 1
        hand_made_strings[f'round{round_index}'].append(generated_string)
            

In [51]:
generate_random_strings(hand_made_strings, 0, 2)
generate_random_strings(hand_made_strings, 1000, 3)
generate_random_strings(hand_made_strings, 2000, 4)
generate_random_strings(hand_made_strings, 3000, 5)

In [52]:
hand_made_strings = {'round1': ['ACB', 'CCADD', 'CADEA', 'ABCDEF', 'FEADE'],
 'round2': ['FEC', 'AFEBC', 'FFAAF', 'BDCDD', 'BACAAC'],
 'round3': ['EEA', 'EAEFA', 'DCBAC', 'CCECC', 'CFDEDB'],
 'round4': ['CFC', 'DCADF', 'EDEDF', 'AFEBE', 'DEBCED'],
 'round5': ['DBB', 'BCCBB', 'BDCAC', 'DEEDF', 'CBAFDE']}

{'round1': ['ACB', 'CCADD', 'CADEA', 'ABCDEF', 'FEADE'],
 'round2': ['FEC', 'AFEBC', 'FFAAF', 'BDCDD', 'BACAAC'],
 'round3': ['EEA', 'EAEFA', 'DCBAC', 'CCECC', 'CFDEDB'],
 'round4': ['CFC', 'DCADF', 'EDEDF', 'AFEBE', 'DEBCED'],
 'round5': ['DBB', 'BCCBB', 'BDCAC', 'DEEDF', 'CBAFDE']}

In [1]:
import pickle

# with open('Reactor_puzzle_strings.pkl', 'wb') as file:
#     pickle.dump(hand_made_strings, file)

with open('../data/Reactor_puzzle_strings.pkl', 'rb') as file:
    hand_made_strings = pickle.load(file)
    
hand_made_strings

{'round1': ['ACB', 'CCADD', 'CADEA', 'FEADE', 'ABCDEF'],
 'round2': ['FEC', 'AFEBC', 'FFAAF', 'BDCDD', 'BACAAC'],
 'round3': ['EEA', 'EAEFA', 'DCBAC', 'CCECC', 'CFDEDB'],
 'round4': ['CFC', 'DCADF', 'EDEDF', 'AFEBE', 'DEBCED'],
 'round5': ['DBB', 'BCCBB', 'BDCAC', 'DEEDF', 'CBAFDE']}